# Building RAG (LangChain) Using Yi's API

In this tutorial, we will learn how to build a Retrieval-Augmented Generation (RAG) system using LangChain and Yi's API.

## Installing Necessary Libraries

First, we need to install the LangChain library.

In [ ]:
!pip install langchain

## Setting Up Environment Variables

Next, we need to configure the LangSmith and Yi API keys. Please ensure you have registered and obtained API keys from [LangSmith](https://smith.langchain.com/) and [Yi Open Platform](https://platform.01.ai/apikeys).

In [ ]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "your_langsmith_api_key"
os.environ["YI_API_KEY"] = "your_yi_api_key"

## Installing LangChain-OpenAI Library

We also need to install the `langchain-openai` library.

In [ ]:
!pip install -qU langchain-openai

## Configuring the LLM

Now let's configure the LLM using Yi's large language model.

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url="https://api.01.ai/v1",
    api_key=os.environ["YI_API_KEY"],
    model="yi-large",
)

## Loading Data

Next, we will load some sample data. Here, we use LangChain's WebBaseLoader to load web page data.

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

## Building a Vector Database

We will use HuggingFace's embedding model to build a vector database and use Chroma to store the vectors.

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load the embedding model
embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")

# Split the documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Build the vector database
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding)
retriever = vectorstore.as_retriever()

## Building the RAG Chain

Finally, we will build the RAG chain and use LangChain's hub to get the prompt.

In [ ]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = rag_chain.invoke("What is Task Decomposition?")
print(response)